## Object: this script is used to eliminate ROIs out of the margine. 

In [1]:
import cv2
import os,sys
import platform as plat
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import heapq
import csv
from os import listdir
from os.path import isfile, join
import seaborn as sns
import numpy as np
import sklearn.decomposition as DECO
from scipy.fftpack import fft, ifft
from scipy.ndimage import gaussian_filter
import pandas as pd
from scipy.signal import find_peaks
import  scipy.io as io
import h5py
import xlrd
import xlwt
import heapq
import maskdb_parsing as maskdb

In [2]:
flg_delete_notlow_active_cell_idx = 1

# Functions ......................................
def categories(x, threshold = .1):
    sub_set = []
    sd = np.std(x)
    md = np.mean(x)
    list_inactive = [a for a in x if a < threshold]
    return list_inactive

class data_input:
    def __init__(self, data):
        self.df = data['dff']
        self.coor = data['coord']
        self.activity_lvl = data['coord'][:, 0] * 0
        self.ACTIVESNESS = data['coord'][:, 0] * 0

def normalized(x):
    return (x - min(x)) / (max(x) - min(x))

        

class data_DF_COOR_MASK:
    def __init__(self, dff, coor, mask):
        self.df = dff
        self.coor = coor
        self.masks = mask

def anatomical_masking (coor):
    mdb = maskdb.mask_db()
    coor = coor [:,[2,1,0]]
    anatomical_mask = []
    for i in range(294):
        anatomical_mask_T = mdb.mask_multi_direct_search(i, coor)[0]
        for i in range(len(anatomical_mask_T)):
            if anatomical_mask_T[i] == False:
                anatomical_mask_T[i] = 0
            if anatomical_mask_T[i] == True:
                anatomical_mask_T[i] = 1
        anatomical_mask.append(anatomical_mask_T)
    return anatomical_mask

def find_index(M, cind):
    ind = []
    for item in range( len( M)):
        if M[item] == cind:
            ind.append( item )
    return ind       

In [3]:
df_behavior = pd.read_excel('/Users/jialexu/Library/CloudStorage/Box-Box/UCSF/Research/Project8LD_NeuralNetwork/experiments/behavior_test/DerivedData/SDA_VDA_selection_20220427.xlsx',
                            sheet_name=4, index_col=0)
df_behavior

,Fish_ID,Mean_LDCI,Cat
Image_ID,,,
Fish06,20220427-Plate1-5C,-0.995255,SDA
Fish03,20220427-Plate1-5D,-0.988801,SDA
Fish09,20220427-Plate1-9C,-0.987146,SDA
Fish11,20220427-Plate1-1B,-0.980678,SDA
Fish13,20220427-Plate1-6D,-0.979944,SDA
Fish01,20220427-Plate1-3E,-0.968221,SDA
Fish12,20220427-Plate1-2G,-0.266682,VDA
Fish02,20220427-Plate1-10F,-0.203456,VDA
Fish04,20220427-Plate1-5G,-0.139854,VDA


In [4]:
FishID='11'
wdir = '/Users/jialexu/Library/CloudStorage/Box-Box/UCSF/Research/Project8LD_NeuralNetwork/experiments/caiman/20220427/Fish'+FishID
DFF_folder = wdir +'/dff/'
results_folder = wdir +'/result/'
result_dir = '/Users/jialexu/Library/CloudStorage/Box-Box/UCSF/Research/Project8LD_NeuralNetwork/experiments/caiman/20220427/All_fish/'


# Loading the data

In [5]:
'''
Reading DF/F from one file: DFF_folder
Reading coordinates from another file:Coord_folder 
 DF/F : all_dta_fsh_dff
 Coordinate: all_dta_fsh_coor

   >>> output: dta_fsh <<<<

'''
#make a list of file names for df/f files generated from caiman
all_input_data_file_dff = []
for item in sorted(os.listdir(DFF_folder)):
    if not item.startswith('.') and os.path.isfile(os.path.join(DFF_folder, item)):
        all_input_data_file_dff.append(item)

        

In [6]:
# stack all df/f files in one object

all_dta_fsh_dff = [np.load(DFF_folder + str(all_input_data_file_dff[file_idx]), allow_pickle=True)
               for file_idx in range (len(all_input_data_file_dff))]


# combine both file stacks for each fish
dta_fsh = []
for fil_idx in range(len(all_dta_fsh_dff)):
    dta_fsh.append(data_input(all_dta_fsh_dff[fil_idx])) 

## Marginal cells removal


In [7]:
# Define the marginal threshold ........................
eps_margine_y= 0
eps_margine_x= 0

# find coordinates of ROIs outside the margine in each zplane
marginal_cell_idx = []
print ("find ROIs out of margine in %i zplanes"%len(all_dta_fsh_dff))
for fish_id in range(len(all_dta_fsh_dff)):
    
    
    marginal_idx = []
    for m_idx in range(len(dta_fsh[fish_id].coor)):
        if dta_fsh[fish_id].coor [m_idx, 1] < 20 + eps_margine_y or \
        dta_fsh[fish_id].coor [m_idx, 1] > 492 - eps_margine_y or\
        dta_fsh[fish_id].coor [m_idx, 0] < 20 + eps_margine_x or \
        dta_fsh[fish_id].coor [m_idx, 0] > 1004 - eps_margine_x:
            marginal_idx.append(m_idx)
    marginal_cell_idx.append(marginal_idx)


find ROIs out of margine in 28 zplanes


In [8]:
# Deleting marginal cells
dta_fsh_org_df = []
dta_fsh_org_coor = []

for fish_id in range(len(dta_fsh)):
    dta_fsh_org_df.append(dta_fsh[fish_id].df)
    dta_fsh_org_coor.append(dta_fsh[fish_id].coor)
    dta_fsh[fish_id].df = np.delete(dta_fsh[fish_id].df, marginal_cell_idx[fish_id], 0)
    dta_fsh[fish_id].coor = np.delete(dta_fsh[fish_id].coor, marginal_cell_idx[fish_id], 0)
    dta_fsh[fish_id].activity_lvl = np.delete(dta_fsh[fish_id].activity_lvl, marginal_cell_idx[fish_id], 0)

print ('no of ROI in each zplane after margin removal')
for stid in range(len(dta_fsh)):
    print ((stid, len(dta_fsh[stid].coor)))

no of ROI in each zplane after margin removal
(0, 4690)
(1, 8809)
(2, 5684)
(3, 6837)
(4, 5739)
(5, 8657)
(6, 8698)
(7, 9060)
(8, 4826)
(9, 9068)
(10, 9122)
(11, 9177)
(12, 8795)
(13, 9094)
(14, 3770)
(15, 4177)
(16, 4531)
(17, 8994)
(18, 9085)
(19, 9182)
(20, 9174)
(21, 6705)
(22, 4132)
(23, 3282)
(24, 9000)
(25, 2783)
(26, 6818)
(27, 9072)


In [ ]:
# Saving output to a .NPZ file
for stack_id in range(len(all_input_data_file_dff)):
    np.savez (results_folder + 'clean_data/20220501/' +  "cleaned_H2BGCaMP_" + os.path.basename(all_input_data_file_dff[stack_id][:-4]) ,
              dff = dta_fsh [stack_id].df,
              coord = dta_fsh [stack_id].coor)
    

In [ ]:
##add z coordinates to ROIs in each zplane
##convert units of coordinates from um to pixel
##save the output dataset in csv file for ANTs to transform
pix_size = 0.977
num_zplanes = 28
zstep = 12
new_rate = 1

coordz_for_ants=[];
num_zplanes = len(dta_fsh)
Zs = range(0,num_zplanes)
for z in Zs:

    A = dta_fsh [z].coor
    C = dta_fsh [z].df
    
    for i,c in zip(A,C):
        coordz_for_ants.append((i[0]*pix_size,i[1]*pix_size,((z*zstep))))
np.savetxt(results_folder + "Adjusted_output/20220501/" +  "Fish"+FishID+'_for_ants_coor_transform' +'.csv', 
        coordz_for_ants, header ='x,y,z', delimiter=",", fmt='%.8f',comments='') 

## Add mask on each ROI

In [9]:
msk_name = np.load ("MaskDatabaseNames.npz")['arr_0']
dta_folder = results_folder+'Adjusted_output/20220501/'
data_coor_ants = np.loadtxt (dta_folder + 'Fish'+FishID+'_for_ants_coor_registered.csv',skiprows=1, delimiter=',')
df_adrss = results_folder+'clean_data/20220501/' 
final_results_folder  = results_folder+"final_result/20220501"


In [10]:
'''

dff comes from caiman npz file


'''

dff_coor_dta = []
for item in sorted(os.listdir(df_adrss)):
    if not item.startswith('.') and os.path.isfile(os.path.join(df_adrss, item)):
        dff_coor_dta.append(item)
dff_coor_dta

['cleaned_H2BGCaMP_Fish11_00.npz',
 'cleaned_H2BGCaMP_Fish11_01.npz',
 'cleaned_H2BGCaMP_Fish11_02.npz',
 'cleaned_H2BGCaMP_Fish11_03.npz',
 'cleaned_H2BGCaMP_Fish11_04.npz',
 'cleaned_H2BGCaMP_Fish11_05.npz',
 'cleaned_H2BGCaMP_Fish11_06.npz',
 'cleaned_H2BGCaMP_Fish11_07.npz',
 'cleaned_H2BGCaMP_Fish11_08.npz',
 'cleaned_H2BGCaMP_Fish11_09.npz',
 'cleaned_H2BGCaMP_Fish11_10.npz',
 'cleaned_H2BGCaMP_Fish11_11.npz',
 'cleaned_H2BGCaMP_Fish11_12.npz',
 'cleaned_H2BGCaMP_Fish11_13.npz',
 'cleaned_H2BGCaMP_Fish11_14.npz',
 'cleaned_H2BGCaMP_Fish11_15.npz',
 'cleaned_H2BGCaMP_Fish11_16.npz',
 'cleaned_H2BGCaMP_Fish11_17.npz',
 'cleaned_H2BGCaMP_Fish11_18.npz',
 'cleaned_H2BGCaMP_Fish11_19.npz',
 'cleaned_H2BGCaMP_Fish11_20.npz',
 'cleaned_H2BGCaMP_Fish11_21.npz',
 'cleaned_H2BGCaMP_Fish11_22.npz',
 'cleaned_H2BGCaMP_Fish11_23.npz',
 'cleaned_H2BGCaMP_Fish11_24.npz',
 'cleaned_H2BGCaMP_Fish11_25.npz',
 'cleaned_H2BGCaMP_Fish11_26.npz',
 'cleaned_H2BGCaMP_Fish11_27.npz']

In [11]:
# dff and coord        
all_dta_fsh = [np.load(df_adrss + str(dff_coor_dta[file_idx]), allow_pickle=True)
               for file_idx in range (len(dff_coor_dta))]
data_dff_tmp = []
for fil_idx in range(len(all_dta_fsh)):
    data_dff_tmp.append(all_dta_fsh[fil_idx]['dff']) 

In [12]:
'''
Merging all dff files as one variable
'''
data_dff = data_dff_tmp[0]
for i in range(1, len(data_dff_tmp)):
    data_dff =np.vstack ([data_dff , data_dff_tmp[i]]) 

#check if neuron number is the same in coordinate and dff file
if np.shape(data_coor_ants)[0] == np.shape(data_dff)[0]:
    print('neuron number in coordinate file is equal to that in dff file: %i'%np.shape(data_coor_ants)[0])

neuron number in coordinate file is equal to that in dff file: 198961


In [13]:
# ---------------------------------------------------------#
#         Assiging the anatomical label  - Registred data  #
# ---------------------------------------------------------#
# Convert "Mu" to "PIXEL"
data_coor_ants [:, 0] = [int(data_coor_ants [i, 0]*1.2531) for i in range(len(data_coor_ants))] # 
data_coor_ants [:, 1] = [int(data_coor_ants [i, 1]*1.2531) for i in range(len(data_coor_ants))]
data_coor_ants [:, 2] = [int(data_coor_ants [i, 2]*.5) for i in range(len(data_coor_ants))]

In [14]:
# CROPPING  -- exlude the extreme border hence keep 137 620 1405
x_min = 0 # 191
x_max = 1405 # 619
y_min = 0 # 35
y_max = 620 # 409
z_min = 0 # 0
z_max = 137 # 274
roi2del = []
for i in range(len(data_coor_ants)):
    if data_coor_ants[i,0] < x_min or \
    data_coor_ants[i,0] > x_max or \
    data_coor_ants[i,1] < y_min or \
    data_coor_ants[i,1] > y_max or \
    data_coor_ants[i,2] < z_min or\
    data_coor_ants[i,2] > z_max:
        roi2del.append(i)
data_coor_ants_crop = np.delete (data_coor_ants ,roi2del , 0)
data_dff_crop = np.delete (data_dff ,roi2del , 0)

print("%i neuron out of the margine were cropped"%len(roi2del))

24734 neuron out of the margine were cropped


In [26]:
data_coor_ants_crop[:]

array([[1269.,  429.,    0.],
       [1272.,  432.,    0.],
       [1286.,  426.,    0.],
       ...,
       [ 217.,  619.,  136.],
       [ 308.,  618.,  137.],
       [ 346.,  618.,  137.]])

In [ ]:
## import data of anatomical_masking
## assign mask index to each ROI
mask = anatomical_masking ( data_coor_ants_crop[:] )

anatomical_mask_MC = np.reshape (mask , [np.shape(mask)[0], np.shape(mask)[1]])

In [ ]:
## assign mask name as the index of the dataset
## in the resulted dataframe, each row is for one mask and each column is for one ROI. 
## cell values 1 means the ROI is assigned to the corresponding mask
df_anatomical_mask = pd.DataFrame(anatomical_mask_MC)
df_msk_name = pd.DataFrame(msk_name)
df_msk_name[0] = df_msk_name[0].str.decode("utf-8").apply(lambda x: x.split("'")).str[1]
df_anatomical_mask.index = df_msk_name[0]
df_anatomical_mask.index.name=None
df_anatomical_mask

In [ ]:
##exclude ROIs that are inactive all the time in dff file and ROIs in eye region
df_eye_neuron = df_anatomical_mask.loc['Ganglia - Eyes'].transpose()
eye_index = df_eye_neuron[df_eye_neuron==1].dropna().index
inactive_neuron = df_anatomical_mask.loc[:,df_anatomical_mask.sum()!=0].columns
print("%i ROIs are excluded due to inactiveness"%(np.shape(data_dff_crop)[0]-len(inactive_neuron)))
print("%i ROIs in eye regions are excluded"%(len(eye_index)))
df_dff_active = pd.DataFrame(data_dff_crop).iloc[inactive_neuron]
df_dff_active = df_dff_active.drop(eye_index)


##exclude ROIs that are inactive all the time in mask file
df_anatomical_mask_active = df_anatomical_mask.iloc[:, df_anatomical_mask.loc[:,df_anatomical_mask.sum()!=0].columns]
df_anatomical_mask_active = df_anatomical_mask_active.drop(eye_index, axis=1)

##exclude ROIs that are inactive all the time in coor file
df_coor_ants_active = pd.DataFrame(data_coor_ants_crop).iloc[df_anatomical_mask.loc[:,df_anatomical_mask.sum()!=0].columns]
df_coor_ants_active = df_coor_ants_active.drop(eye_index)



In [ ]:
df = df_dff_active.loc[df_coor_ants_active.sort_values(by=2).index]
def normalization(row):
    mean = row.mean()
    std = row.std()
    norm1 = (row-mean)/std
    norm2 = (norm1-norm1.min())/(norm1.max()-norm1.min())
    return norm2
df_normalized = df.apply(normalization,axis=1).dropna()

In [ ]:
df_normalized

In [ ]:
f, ax=plt.subplots(figsize=(100,50))
cmap = sns.diverging_palette(230, 10, as_cmap=True)
sns.heatmap(df_normalized.round(2), center=0,cmap=cmap,\
            square=False, linewidths=0, cbar_kws={"shrink": 0.9,'label':'activity'}, \
            annot=None,annot_kws={'fontsize':12}, ax=ax)
ax.set_title('Fish'+FishID + ': '+ df_behavior.loc[('Fish'+FishID)]['Cat']+\
             ' LDCI = ' +str(df_behavior.loc[('Fish'+FishID)]['Mean_LDCI'].round(4)), \
             size=100)

plt.savefig(result_dir+'Fish'+FishID+'.png')
plt.show()

In [ ]:
df_dff_active.to_csv(result_dir+'Fish'+FishID+'_dff.csv')
df_anatomical_mask_active.to_csv(result_dir+'Fish'+FishID+'_final_mask.csv')
df_coor_ants_active.to_csv(result_dir+'Fish'+FishID+'_coor.csv')


In [ ]:
f = plt.figure(figsize=(20,10))
plt.scatter(df_coor_ants_active.iloc[:, 0], df_coor_ants_active.iloc[:, 1],
            c = df_dff_active.mean(axis=1), cmap='Reds')
plt.title('Fish'+FishID+' '+str(df_dff_active.shape[0])+' neuron ', size = 15)
plt.show()